## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Нам предоставили таблицу с данными о заемщиках банка, в которую занесена основная информация, такая как пол, возраст, семейное положение, количество детей, образование и так далее.

Опираясь на эти данные, нам требуется ответить на несколько вопросов:

<dd> * Есть ли зависимость между наличием детей и возвратом кредита в срок?</dd>
<dd> * Есть ли зависимость между семейным положением и возвратом кредита в срок?</dd>
<dd> * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</dd>
<dd> * Как разные цели кредита влияют на его возврат в срок?</dd>

Прежде чем приступить к анализу данных, нам необходимо проверить их на наличие ошибок. Собранные данные могут иметь недопустимые значения, несогласующаяся между собой информация, отсутствующие значения и т.д. Поэтому первоначальный этап - предобработка данных. На этом этапе мы выявим все артефакты, отсутствующие или дублирующиеся значения, обдумаем причины их появления и решим как с ними поступить.


### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') #загрузка файла с данными
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод


Полученный файл содержит 12 столбцов с данными о заемщиках банка. Видим, что данные еще не готовы к анализу: столбец с общим трудовым стажем в днях должен быть целочисленным и иметь только положительные значения, записи в столбце с уровнем образования клиента имеют разный регистр.

Приступим ко второму шагу - предобработке данных, чтобы выявить все недочеты предоставленных данных и исправить их.




### Шаг 2. Предобработка данных

### Обработка пропусков


В данном разделе мы должны выявить пропущенные значения в переданных данных и принять решение: удалить пропущенные значения или же заполнить их нулями, средним арифметическим значением либо медианой.


In [3]:
#определим столбцы с неопределенными значениями и общее число пропусков в этих столбцах
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Видим, что в столбцах _days_employed_ и _total_income_ равное количество пропусков. Таким образом, если мы определим, что эти значения в этих столбцах отсутствуют одновременно, это может означать, что конкретный человек не работает и пропуски в данном случае можно заменить на нули.


In [4]:
#в случаях, если оба столбца имеют неопределенные значения, заполняем их нулями
data.loc[(data['days_employed'].isnull())&(data['total_income'].isnull()), ('days_employed', 'total_income')] = 0
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод


На этом этапе мы выявили пропущенные значения и заменили их на нули.




### Замена типа данных


При просмотре информации из файла данных, мы увидели, что столбец с информацией о трудовом стаже имеет отрицательные значения. Возможно, и другие столбцы тоже имеют такие данные. Это может быть связано с тем, что при чтении информации и записи ее в файл символ "тире" был заменен на знак "минус", поэтому такие значения мы должны брать по модулю. Также, для удобства, создадим новый столбец _years_employed_, который бы содержал информацию о стаже в годах.


In [6]:
data[data['days_employed']<0]['days_employed'].describe() #посмотрим описание данных по отрицательному стажу

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

In [7]:
data[data['days_employed']>0]['days_employed'].describe() #и по положительному

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64


Что удивительно, отрицательные значения трудового стажа, помимо знака, выглядят совершенно правдоподобно, тогда как положительные в несколько раз превышают допустимые значения. Согласно ТК РФ официально работать можно с 16 лет, здесь же значения гораздо выше возможных.

Проанализируем, у кого встречаются эти значения.


In [9]:
data[data['days_employed']>data['dob_years']-16]['income_type'].value_counts()

пенсионер      3446
сотрудник         5
компаньон         2
безработный       2
Name: income_type, dtype: int64

In [10]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


Почти все пенсионеры и все безработные неправильно заполнили графу с трудовым стажем в днях. Просто пренебречь этими данными мы не можем, т.к. на выходе получим недостоверные результаты, отбросив такое количество значений.

Попробуем изучить информацию о возрасте пенсионеров, неправильно заполнивших данные о стаже.


In [11]:
data[(data['days_employed']>data['dob_years']-16)&(data['income_type']=='пенсионер')]['dob_years'].mean()

59.08560650029019


Теперь проверим какой средний стаж может быть у человека. 59 лет - 16 лет = 43 года. То есть 15695 дней, в то время как средний указанный стаж (ошибочный) аж 365004 дня. Конечно, считаем мы очень усредненно, но тем не менее это почти в 30 раз больше!

На самом деле, это число - 30 - наводит на определенные мысли. Возможно, что человек переводил свой стаж в годах в дни, умножил число на 365, а потом подумал: "Так, мне же нужно в днях, я посчитал количество месяцев, теперь умножим это значение на число дней в месяце - на 30".

Трудно сказать точно с чем была связана эта ошибка, нужно подумать, как ее исправить, если этими строками мы пренебречь не можем.

Как вариант, можем вместо завышенных значений вставить средние. Вопрос в том, какие средние значения использовать. Логичнее было бы использовать средние значения по тому же типу занятости, но мы не можем заполнить 3000 записей по среднему значению из четырехста оставшихся.

Поэтому заполним некорректные значения средними по разным возрастным группам.


In [12]:
year_data = data.loc[(data['days_employed']<0)&(data['dob_years']!=0)].groupby('dob_years')['days_employed'].mean()

In [13]:
summa_25 = 0 #посчитаем общую сумму стажа и количество по разным возрастным группам
count_25 = 0
summa_35 = 0
count_35 = 0
summa_45 = 0
count_45 = 0
summa_55 = 0
count_55 = 0
summa_64 = 0
count_64 = 0
summa_65 = 0
count_65 = 0
for i in range(len(year_data)):
    if 0<i<6:
        summa_25 += year_data[i+19]
        count_25 += 1
    elif 6<=i<16:
        summa_35 += year_data[i+19]
        count_35 += 1
    elif 16<=i<26:
        summa_45 += year_data[i+19]
        count_45 += 1
    elif 26<=i<36:
        summa_55 += year_data[i+19]
        count_55 += 1
    elif 36<=i<46:
        summa_64 += year_data[i+19]
        count_64 += 1
    else:
        summa_65 += year_data[i+19]
        count_65 += 1

In [14]:
for row in range(len(data)): #в наших данных заменим положительные значения на средние по возрастным группам
    if data['days_employed'][row]>0:
        if data['dob_years'][row]<25:
            data.loc[row, 'days_employed'] = summa_25 / count_25
        elif 25<=data['dob_years'][row]<35:
            data.loc[row, 'days_employed'] = summa_35 / count_35
        elif 35<=data['dob_years'][row]<45:
            data.loc[row, 'days_employed'] = summa_45 / count_45
        elif 45<=data['dob_years'][row]<55:
            data.loc[row, 'days_employed'] = summa_55 / count_55
        elif 55<=data['dob_years'][row]<65:
            data.loc[row, 'days_employed'] = summa_64 / count_64
        else:
            data.loc[row, 'days_employed'] = summa_65 / count_65
    else:
        pass

In [15]:
data['days_employed'].describe()

count    21525.000000
mean     -2301.436233
std       2172.065860
min     -18388.949901
25%      -3657.812023
50%      -1799.549893
75%       -610.652074
max          0.000000
Name: days_employed, dtype: float64


Мы избавились от некорректных значений. Теперь сделаем все значения трудового стажа положительными.


In [16]:
data['days_employed'] = abs(data['days_employed'])
data['children'] = abs(data['children'])
data['days_employed'] = data['days_employed'].astype('int')


Также нам необходимо изменить регистр в столбце о высшем образовании и привести его к одному, иначе могут возникнуть ошибки при анализе данных.


In [17]:
data['education_lower'] = data['education'].str.lower()
data['education_lower'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lower, dtype: int64

In [18]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Оценивая информацию о количестве детей, мы видим значения, вероятнее всего, не соответствующие истинным - некоторые люди ввели в этом пункте значение 20. Трудно сказать как правильнее всего поступить с такими значениями. Скорее всего, если в этом пункте стояло значение "2.", это могло быть расценено при машинной обработке как "20", поэтому исправим эти значения в соответствии с этим предположением.


In [19]:
data.loc[data['children']==20, 'children'] = 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод


В этом пункте мы заменили тип данных у столбца с трудовым стажем, а также обработали некоторые аномальные значения данных. Можем приступить к следующему пункту - обработке дублирующихся значений.




### Обработка дубликатов


Появляющиеся в данных дубликаты, вероятнее всего связаны с человеческим фактором, когда человек повторно обрабатывает ту же анкету, которую, например, до него уже обработал кто-то еще.


In [20]:
data.duplicated().sum() #проверяем количество дублирующихся строк

54

In [21]:
data = data.drop_duplicates()
data = data.dropna().reset_index(drop=True)
data.duplicated().sum()

0

### Вывод


Итак, предобработка данных завершена. Проведен общий анализ переданной информации, удалены дублирующиеся строки, заменены пропуски. Можем приступить к следующему этапу работы.




### Лемматизация


Лемматизация - это приведение слова к его словарной форме - лемме. В данном случае мы можем привести к леммам цели получения кредита. 


In [22]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_category'] = data['purpose']
#просмотрим столбец с целями кредита для выделения лемм

data['purpose_category'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      


Мы можем выделить несколько целей кредита - "образование", "недвижимость", "автомобиль", "семья".


In [23]:
data['purpose_category'] = data['purpose'] #выделяем леммы в зависимости от цели кредита
for row in range(len(data)):
    lemma = m.lemmatize(data['purpose'][row])
    if 'жилье' in lemma:
        data.loc[row, 'purpose_category'] = 'недвижимость'
    elif 'недвижимость' in lemma:
        data.loc[row, 'purpose_category'] = 'недвижимость'
    elif 'ремонт' in lemma:
        data.loc[row, 'purpose_category'] = 'недвижимость'
    elif 'образование' in lemma:
        data.loc[row, 'purpose_category'] = 'образование'
    elif 'обучение' in lemma:
        data.loc[row, 'purpose_category'] = 'образование'
    elif 'свадьба' in lemma:
        data.loc[row, 'purpose_category'] = 'семья'
    elif 'семья' in lemma:
        data.loc[row, 'purpose_category'] = 'семья'
    elif 'автомобиль' in lemma:
        data.loc[row, 'purpose_category'] = 'автомобиль'
    elif 'машина' in lemma:
        data.loc[row, 'purpose_category'] = 'автомобиль'
    else:
        data.loc[row, 'purpose_category'] = 'другое'

In [24]:
data['purpose_category'].value_counts() #проверка полученных значений

недвижимость    10814
автомобиль       4308
образование      4014
семья            2335
Name: purpose_category, dtype: int64

### Вывод


Таким образом, все цели получения кредита мы разделили на четыре категории: недвижимость, автомобиль, образование и семья.




### Категоризация данных


Среди данных у нас уже выделено несколько категорий: это столбец с образованием - высшее, среднее и пр., столбец с семейным положением - женат, разведен и пр., пол клиента также имеет ограниченный набор значений, столбец _debt_ также является категоризированным.

Создадим еще один столбец, в котором разделим уровень дохода на несколько категорий.


In [26]:
data['prestige'] = pd.qcut(data[data['total_income']!=0]['total_income'], 5) #разделим всех заемщиков на 5 категорий по типу дохода
data['prestige'].head()

0    (223107.554, 2265604.029]
1       (94836.83, 128279.674]
2     (128279.674, 164660.939]
3    (223107.554, 2265604.029]
4     (128279.674, 164660.939]
Name: prestige, dtype: category
Categories (5, interval[float64]): [(20667.263, 94836.83] < (94836.83, 128279.674] < (128279.674, 164660.939] < (164660.939, 223107.554] < (223107.554, 2265604.029]]


Таким образом мы смогли выделить пять одинаковых групп по уровню дохода, поэтому теперь мы сможем категоризовать заемщиков по их доходу более равномерно.


In [27]:
def groupe_income(count):
    if 0< count <= 94836:
        return 'низкий'
    elif 94836 < count <= 128279:
        return 'ниже среднего'
    elif 128279 < count <= 164660:
        return 'средний'
    elif 164660 < count <= 223107:
        return 'выше среднего'
    elif count > 223107:
        return 'высокий'
    else:
        return 'нет'
data['income_id'] = data['total_income'].apply(groupe_income)
data['income_id'].value_counts()

высокий          3871
ниже среднего    3870
средний          3870
выше среднего    3870
низкий           3870
нет              2120
Name: income_id, dtype: int64



### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
data_grouped_child = data.groupby('children').agg({'debt': ['count', 'sum']})
data_grouped_child['conversion'] = data_grouped_child[('debt', 'sum')]/data_grouped_child[('debt', 'count')]*100
data_grouped_child.sort_values(by='conversion', ascending=False)

debt       conversion
          count   sum           
children                        
4            41     4   9.756098
2          2128   202   9.492481
1          4856   445   9.163921
3           330    27   8.181818
0         14107  1063   7.535266
5             9     0   0.000000

In [29]:
sum_child = 0
count_child = 0
sum_childfree = 0
count_childfree = 0
for row in range(len(data_grouped_child)):
    if row == 0:
        sum_childfree = data_grouped_child[('debt', 'sum')][row]
        count_childfree = data_grouped_child[('debt', 'count')][row]
    else:
        sum_child += data_grouped_child[('debt', 'sum')][row]
        count_child += data_grouped_child[('debt', 'count')][row]
print('Доля невозврата для людей с детьми:', sum_child/count_child*100, 'без детей:', sum_childfree/count_childfree*100)

Доля невозврата для людей с детьми: 9.206952743074416 без детей: 7.535266179910682


### Вывод


В целом люди без детей чаще возвращают кредит в срок, чем люди с детьми, хотя разница невелика. Доля невозврата кредита для людей без детей 7,5% против 9,2% для людей с детьми.




- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
data_grouped_family = data.groupby('family_status_id').agg({'debt': ['count', 'sum']})
data_grouped_family['conversion'] = data_grouped_family[('debt', 'sum')]/data_grouped_family[('debt', 'count')]*100
data_grouped_family.sort_values(by='conversion', ascending=False)

debt      conversion
                  count  sum           
family_status_id                       
4                  2810  274   9.750890
1                  4163  388   9.320202
0                 12344  931   7.542126
3                  1195   85   7.112971
2                   959   63   6.569343

### Вывод


Неженатые клиенты и клиенты в гражданском браке гораздо реже возвращают кредит в срок. А самые надежные заемщики - вдовцы. Возможно, семейное положение имеет значение в данном вопросе: женатые и замужние более ответственные и благонадежные.




- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
data_grouped_income = data[data['total_income']!=0].groupby('income_id').agg({'debt': ['count', 'sum']})
data_grouped_income['conversion'] = data_grouped_income[('debt', 'sum')]/data_grouped_income[('debt', 'count')]*100
data_grouped_income.sort_values(by='conversion', ascending=False)

debt      conversion
              count  sum           
income_id                          
средний        3870  335   8.656331
выше среднего  3870  329   8.501292
ниже среднего  3870  325   8.397933
низкий         3870  308   7.958656
высокий        3871  274   7.078274

### Вывод


Доля невозврата в данном случае имеет примерно одинакокое значение. Но с небольшим отрывом лучшие заемщики - люди с высоким уровнем дохода.




- Как разные цели кредита влияют на его возврат в срок?

In [32]:
data_grouped_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})
data_grouped_purpose['conversion'] = data_grouped_purpose[('debt', 'sum')]/data_grouped_purpose[('debt', 'count')]*100
data_grouped_purpose.sort_values(by='conversion', ascending=False)

debt      conversion
                  count  sum           
purpose_category                       
автомобиль         4308  403   9.354689
образование        4014  370   9.217738
семья              2335  186   7.965739
недвижимость      10814  782   7.231367

### Вывод


Доля невозврата для цели "недвижимость" и "семья" явно ниже. Возможно, новое образование может разочаровать и не принести планируемого дохода, а содержание автомобиля само по себе очень дорогое удовольствие. Возможно, люди не ожидают, что траты на автомобиль будут такими высокими и не могут справиться с ними и еще и с кредитом.




### Шаг 4. Общий вывод


Мы проанализировали несколько зависимостей надежности заемщиков от целей кредита, доходов, наличия детей и семейного статуса.
Наилучшая зависимость наблюдается с семейным статусом - неженатые люди чаще делают просрочки по кредитам, в отличие от женатых. Также у заемщиков, берущих кредиты на покупку своего жилья, доля невозврата ниже, чем у берущих кредит на покупку автомобиля.
